In [1]:
import pandas as pd

In [2]:
from inc import stop_words as sw
from inc import extra_stopwords as esw

In [5]:
import torch
import torchtext
from torchtext.datasets import text_classification
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
NGRAMS = 1
BATCH_SIZE = 16

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
from sqlalchemy import create_engine
table_name = 'Training_set'
db_path = 'db/training.sqlite'
db_path = 'sqlite:///' + db_path
engine = create_engine(db_path, echo=True)

In [9]:
training_data_df = pd.read_sql_table(table_name, engine)

2020-12-16 03:56:10,502 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-16 03:56:10,503 INFO sqlalchemy.engine.base.Engine ()
2020-12-16 03:56:10,505 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-16 03:56:10,506 INFO sqlalchemy.engine.base.Engine ()
2020-12-16 03:56:10,508 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-16 03:56:10,509 INFO sqlalchemy.engine.base.Engine ()
2020-12-16 03:56:10,510 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2020-12-16 03:56:10,511 INFO sqlalchemy.engine.base.Engine ()
2020-12-16 03:56:10,513 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("Training_set")
2020-12-16 03:56:10,514 INFO sqlalchemy.engine.base.Engine ()
2020-12-16 03:56:10,516 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNI

In [10]:
training_data_df = training_data_df.rename(columns={"class": "label"})
training_data_df = training_data_df.convert_dtypes()

In [11]:
training_data_df.dtypes

index        Int64
abstract    string
label        Int64
dtype: object

In [12]:
train = training_data_df.sample(frac=0.8,random_state=200) #random state is a seed value
test = training_data_df.drop(train.index)

In [13]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [15]:
VOCAB_SIZE = len(train.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

AttributeError: 'DataFrame' object has no attribute 'get_vocab'

In [10]:
dataset = Sequences(training_data_df)
train_loader = DataLoader(dataset, batch_size=100)

print(dataset[5][0].shape)

(1, 2326)


In [16]:
len(dataset.token2idx)

2326

In [22]:
class BagOfWordsClassifier(nn.Module):
    def __init__(self, vocab_size, hidden1, hidden2):
        super(BagOfWordsClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 15)
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [23]:
model = BagOfWordsClassifier(len(dataset.token2idx), 128, 64)
model

BagOfWordsClassifier(
  (fc1): Linear(in_features=2326, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=15, bias=True)
)

In [24]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)

In [25]:
import pprint as pp

In [28]:
model.train()
train_losses = []
for epoch in range(10):
    progress_bar = tqdm(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        model.zero_grad()

        output = model(inputs)
        # pp.pprint(target)
        loss = criterion(output.unsqueeze(15), target.float())
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 15)